In [1]:

import math


def haversine(lon1, lat1, lon2, lat2):
    lon1, lat1, lon2, lat2 = map(math.radians, [lon1, lat1, lon2, lat2])
    
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2)**2
    c = 2 * math.asin(math.sqrt(a)) 
    r = 6371
    return c * r

def Estimate_Time(start, end,speed_kmh):
    lon1, lat1 = start['Longitude'], start['Latitude']
    lon2, lat2 = end['Longitude'], end['Latitude']
    
    distance_km = haversine(lon1, lat1, lon2, lat2)
    
    
    time_hours = distance_km / speed_kmh
    time_minutes = time_hours * 60
    
    return {
        'distance_km': round(distance_km, 2),
        'estimated_time_minutes': round(time_minutes, 2)
    }


start = {'Longitude': 2.033, 'Latitude': 36.752}
end = {'Longitude': 3.0588, 'Latitude': 36.7538}
print(Estimate_Time(start, end,49))


{'distance_km': 91.39, 'estimated_time_minutes': 111.91}


In [2]:
def print_pretty_result(result):
    if result['points_visited']<result['Original_points']+1:
        print(f'Visiting All the points In the time limit is unlikely to be optimal. You can try to increase the time limit or average speed.')
    print(f"\n📍 Points Visited: {result['points_visited']}")
    print(f"⏱️ Total Travel Time: {result['total_time_minutes']} minutes")
    print("\n🗺️ Visit Order:")
    for i, point in enumerate(result['visited_points'], start=1):
        name = point.get('name', f"Unnamed ({point['Latitude']}, {point['Longitude']})")
        print(f"  {i}. {name}")

In [3]:
data_points = [
    {'name': 'Point A', 'Longitude': 3.0588, 'Latitude': 36.7538},
    {'name': 'Point B', 'Longitude': 3.062, 'Latitude': 36.75},
    {'name': 'Point C', 'Longitude': 2.944, 'Latitude': 36.766},
    {'name': 'Point D', 'Longitude': 3.1, 'Latitude': 36.76},
    {'name': 'Point E', 'Longitude': 2.889, 'Latitude': 36.712},
    {'name': 'Point F', 'Longitude': 3.11, 'Latitude': 36.765},
    {'name': 'Point G', 'Longitude': 3.15, 'Latitude': 36.755},
    {'name': 'Point H', 'Longitude': 2.95, 'Latitude': 36.775},
    {'name': 'Point I', 'Longitude': 3.07, 'Latitude': 36.78},
    {'name': 'Point J', 'Longitude': 2.97, 'Latitude': 36.79},
    {'name': 'Point K', 'Longitude': 3.05, 'Latitude': 36.74},
    {'name': 'Point L', 'Longitude': 2.93, 'Latitude': 36.72},
    {'name': 'Point M', 'Longitude': 2.91, 'Latitude': 36.71},
    {'name': 'Point N', 'Longitude': 2.88, 'Latitude': 36.70},
    {'name': 'Point O', 'Longitude': 3.16, 'Latitude': 36.77},
    {'name': 'Point P', 'Longitude': 3.19, 'Latitude': 36.76},
    {'name': 'Point Q', 'Longitude': 3.21, 'Latitude': 36.75},
    {'name': 'Point R', 'Longitude': 3.17, 'Latitude': 36.74},
    {'name': 'Point S', 'Longitude': 2.92, 'Latitude': 36.74},
    {'name': 'Point T', 'Longitude': 2.85, 'Latitude': 36.69}
]

In [10]:


from copy import deepcopy


class PlanOptimizerFull(object):
    def __init__(self, data, time_limit, constraints=None, speed=40, daily_limit=480, max_visits_per_day=10):
        self.original_data = deepcopy(data)
        self.data = deepcopy(data)
        self.global_time_limit = time_limit
        self.constraints = constraints
        self.speed = speed
        self.daily_limit = daily_limit
        self.start_point = None
        self.max_visits_per_day = 10

    def dfs(self, current, remaining, path, total_time):
        if total_time > self.daily_limit:
            return
        if len(path) - 1 > self.max_visits_per_day:
            return
        if len(path) > len(self.best_path):
            self.best_path = deepcopy(path)
            self.best_time = total_time

        for i, point in enumerate(remaining):
            travel_time = Estimate_Time(current, point, self.speed)['estimated_time_minutes']
            self.dfs(
                point,
                remaining[:i] + remaining[i+1:],
                path + [point],
                total_time + travel_time
            )

    def optimize(self, start, available_points):
        self.best_path = []
        self.best_time = 0
        self.dfs(start, available_points, [start], 0)
        return {
            'visited_points': self.best_path,
            'total_time_minutes': round(self.best_time, 2),
            'points_visited': len(self.best_path)
        }

    def multi_day_optimize(self, start):
        self.start_point = start
        remaining_points = deepcopy(self.data)
        daily_results = []
        total_time_used = 0

        while remaining_points and total_time_used < self.global_time_limit:
            # Remove already visited points from previous runs
            current_remaining = [
                p for p in remaining_points
                if not (p['Longitude'] == start['Longitude'] and p['Latitude'] == start['Latitude'])
            ]

            day_result = self.optimize(start, current_remaining)
            visited = day_result['visited_points'][1:]  # exclude start point from removal
            daily_time = day_result['total_time_minutes']

            if not visited:
                break  # can't fit any more points

            # Add day result and update
            daily_results.append({
                'day': len(daily_results) + 1,
                'path': day_result['visited_points'],
                'time': daily_time
            })
            total_time_used += daily_time
            remaining_points = [
                p for p in remaining_points
                if p not in visited
            ]

        return daily_results



In [ ]:
start_point = {'name':"Start",'Longitude': 2.9, 'Latitude': 36.75}
optimizer = PlanOptimizerFull(data=data_points, time_limit=480, speed=60)
result = optimizer.optimize(start=start_point,available_points=data_points)

# Call after optimization
print_pretty_result(result)